In [9]:
# MSC. Edwin Calle Condori

import ee
import geemap
import os

# Inicializar la API de Earth Engine
ee.Initialize()

# Definir el AOI como un FeatureCollection desde el Asset
aoi = ee.FeatureCollection('projects/eddycc66/assets/area1')

# Cargar la colección de imágenes Sentinel-2
dataset = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
           .filterBounds(aoi)
           .filterDate('2023-01-01', '2023-12-31')
           .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)))

# Calcular el NDVI para cada imagen en la colección
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

withNDVI = dataset.map(calculate_ndvi)

# Seleccionar las bandas necesarias junto con el NDVI
bands = withNDVI.select(['B4', 'B8', 'B11', 'NDVI'])

# Definir constantes para el cálculo de LST
LST_coeff1 = ee.Number(0.004)
LST_coeff2 = ee.Number(300)

# Calcular la LST aproximada utilizando la banda B11 y el NDVI
def calculate_lst(image):
    LST_image = (image.select('B11')
                 .multiply(LST_coeff1)
                 .add(LST_coeff2)
                 .subtract(image.select('NDVI').multiply(0.01).multiply(273.15))
                 .rename('LST_Celsius'))
    return LST_image.copyProperties(image, image.propertyNames())

LST = bands.map(calculate_lst)

# Calcular la diferencia de temperatura (max - min) para cada pixel
LST_max = LST.max()
LST_min = LST.min()
LST_diff = LST_max.subtract(LST_min).rename('LST_Difference')

# Recortar la imagen al AOI
LST_diff_clipped = LST_diff.clip(aoi.geometry())

# Crear un mapa utilizando geemap (basado en folium)
# Obtener las coordenadas del centro del AOI para centrar el mapa
aoi_centroid = aoi.geometry().centroid().coordinates().getInfo()
map_center = [aoi_centroid[1], aoi_centroid[0]]  # Latitud, Longitud
m = geemap.Map(location=map_center, zoom_start=10)

# Visualizar la diferencia de temperatura en el mapa
LSTDiffVis = {'min': 0, 'max': 20, 'palette': ['blue', 'yellow', 'red']}
m.addLayer(LST_diff_clipped, LSTDiffVis, 'Diferencia de Temperaturas')

# Añadir la leyenda al mapa usando geemap
legend_dict = {
    'Baja': 'blue',
    'Media': 'yellow',
    'Alta': 'red'
}
m.add_legend(legend_title="Diferencia de Temperaturas (°C)", legend_dict=legend_dict)

# Mostrar el mapa
m

# Guardar la imagen recortada en una carpeta local con 10 metros de resolución
output_folder = r"D:\Teledetcccion II\calculoLST"
output_path = os.path.join(output_folder, "LST_Difference_Sentinel2.tif")

# Crear la carpeta si no existe
os.makedirs(output_folder, exist_ok=True)

# Exportar la imagen a disco local con resolución de 10 metros por píxel
geemap.ee_export_image(LST_diff_clipped, filename=output_path, scale=10, region=aoi.geometry(), file_per_band=False)

print(f"Imagen exportada a {output_path}")


Generating URL ...
Please wait ...
Data downloaded to D:\Teledetcccion II\calculoLST\LST_Difference_Sentinel2.tif
Imagen exportada a D:\Teledetcccion II\calculoLST\LST_Difference_Sentinel2.tif
